In [127]:
# loading libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import time
from IPython.display import FileLink

In [241]:
# Defining the target link:
website ='https://www.autobranche.org/category/fahrschulen-verzeichnis/'
response = requests.get(website)
response.status_code

200

In [242]:
soup = BeautifulSoup(response.content, 'html.parser')
results = soup.find_all('article', {'itemtype': 'https://schema.org/CreativeWork'})


In [243]:
len(results)

20

## School_Name

In [244]:
results[0].find('h2', class_='entry-title').get_text(strip=True)

'Fahrschule O & B GmbH'

## Address_Phone

In [133]:
results[0].find('div', class_='entry-summary').get_text(strip=True)

'Beschreibung: Adresse: Rodemannstr. 11 99974 Mühlhausen /Thüringen Telefon: 03601 872286'

## City

In [134]:
City = results[0].find('span', class_='tags-links').get_text(strip=True) 
City_cleaned = City.replace('Schlagwörter', '').strip()
print(City_cleaned)

Mühlhausen /Thüringen


In [135]:
results[0].find('span', class_='tags-links').get_text(strip=True).replace('Schlagwörter', '').strip()



'Mühlhausen /Thüringen'

# Putting Everything together

In [136]:
School_Name = []
Address_Phone = []
City = []

for result in results:
    #School_Name
    try:
        School_Name.append(result.find('h2', class_='entry-title').get_text(strip=True))
    except:
        School_Name.append('n/a')
  

    #Address_Phone
    try:
        Address_Phone.append(result.find('div', class_='entry-summary').get_text(strip=True))
    except:
        Address_Phone.append('n/a')
        
    #City
    try:
        City.append(result.find('span', class_='tags-links').get_text(strip=True).replace('Schlagwörter', '').strip())
    except:
        City.append('n/a')

### Create Pandas Dataframe

In [137]:
# dictionary
Driving_Schools_Spage = pd.DataFrame({'School_Name': School_Name, 'Address_Phone':Address_Phone, 'City':City})

In [245]:
Driving_Schools_Spage.tail()

,School_Name,Address_Phone,City
15,Bernhard Kiel,Beschreibung: Adresse: Pfarrer-Bonhoeffer-Str....,Schlotheim
16,Uwe Rücker,Beschreibung: u. Karsta Fahrschule Adresse: Am...,Mühlhausen /Thüringen
17,Friedrich Schreff,Beschreibung: Adresse: Unterdorf 4 99996 Mente...,Menteroda
18,Hans-Georg Swars,Beschreibung: Adresse: Grünstr. 39 99974 Mühlh...,Mühlhausen /Thüringen
19,Klaus Weidner,Beschreibung: Adresse: Thomas-Müntzer-Str. 13 ...,Ohrdruf


### Pagination

In [139]:
School_Name = []
Address_Phone = []
City = []

for i in range(1, 594):  # We have 593 pages
    
    # Constructing the website URL correctly
    website = 'https://www.autobranche.org/category/fahrschulen-verzeichnis/page/' + str(i) + '/'
    
    # Request to website
    response = requests.get(website)

    # Soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Results
    results = soup.find_all('article', {'itemtype': 'https://schema.org/CreativeWork'})
    
    # Loop through results
    for result in results:
        
        # School_Name
        try:
            School_Name.append(result.find('h2', class_='entry-title').get_text(strip=True))
        except:
            School_Name.append('n/a')
  
        # Address_Phone
        try:
            Address_Phone.append(result.find('div', class_='entry-summary').get_text(strip=True))
        except:
            Address_Phone.append('n/a')
        
        # City
        try:
            City.append(result.find('span', class_='tags-links').get_text(strip=True).replace('Schlagwörter', '').strip())
        except:
            City.append('n/a')
            
   # Pause execution for 2 seconds after scraping each page
    time.sleep(2)


In [140]:
# dictionary
Driving_Schools_Mpages = pd.DataFrame({'School_Name': School_Name, 'Address_Phone':Address_Phone, 'City':City})

In [154]:
Driving_Schools_Mpages.tail(5)

,School_Name,Address_Phone,City
11851,Fahrschule A.Jaensch Inh. Roland Horn,Beschreibung: Adresse: Trachenberger Str. 14 0...,Dresden
11852,"Auguszt, The Driving Instructor",Beschreibung: Fahrschule Adresse: Bautzner Str...,Dresden
11853,Fahrschule Lehmann,Beschreibung: Adresse: Franz-Lehmann-Str. 43 0...,Dresden
11854,Fahrschule Nord Gernot Krakowsky,Beschreibung: Fahrschule Adresse: Bautzner Str...,Dresden
11855,Fahrschule Inh. Glück Michael,Beschreibung: Adresse: Herbststr. 11 01139 Dre...,Dresden


### Cleaning the Dataset

In [247]:
# Renaming the dataframe:
df = Driving_Schools_Mpages.copy()

In [248]:
# Removing the 'Beschreibung: Adresse:' and the 'Beschreibung: Fahrschule Adresse:  from the 'Address_Phone' column:
df['Address_Phone'] = df['Address_Phone'].str.replace('Beschreibung: Adresse:', '', regex=False)
df['Address_Phone'] = df['Address_Phone'].str.replace('Beschreibung: Fahrschule Adresse:', '', regex=False)


In [249]:
# Extracting the telefon number from the 'Address_Phone' column and storing it in a new column:
df['Telefon_Number'] = df['Address_Phone'].str.extract(r'Telefon: (\d[\d\s]+)')

# Extracting the Postal code from the Address_Phone' column and storing it in a new variable:
df['Postal_Code'] = df['Address_Phone'].str.extract(r'(\d{5})')

In [252]:
# Removing the telefon number from the 'Address_Phone' column:
df['Address'] = df['Address_Phone'].str.replace(r'Telefon: (\d[\d\s]+)', '')

C:\Users\Besitzer\AppData\Local\Temp\ipykernel_11748\2784797716.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Address'] = df['Address_Phone'].str.replace(r'Telefon: (\d[\d\s]+)', '')


In [253]:
# Dropng the 'Address_Phone' column:
df.drop('Address_Phone', axis=1, inplace=True)


In [254]:
df

,School_Name,City,Telefon_Number,Postal_Code,Address
0,Fahrschule O & B GmbH,Mühlhausen /Thüringen,03601 872286,99974,Rodemannstr. 11 99974 Mühlhausen /Thüringen
1,Joachim Müller,Körner,036025 50311,99998,Hauptstr. 7 99998 Körner
2,Wicke Norbert,Mühlhausen /Thüringen,03601 873460,99974,Ammerstr. 12 99974 Mühlhausen /Thüringen
3,F & W Fahrschule GmbH,Mühlhausen /Thüringen,03601 812505,99974,Röblingstr. 16 99974 Mühlhausen /Thüringen
4,Michael Hain,Mühlhausen /Thüringen,03601 443024,99974,Weinbergstr. 39 99974 Mühlhausen /Thüringen
...,...,...,...,...,...
11851,Fahrschule A.Jaensch Inh. Roland Horn,Dresden,0351 8480358,01129,Trachenberger Str. 14 01129 Dresden
11852,"Auguszt, The Driving Instructor",Dresden,0351 4906666,01099,Bautzner Str. 56 01099 Dresden
11853,Fahrschule Lehmann,Dresden,0351 8485372,01139,Franz-Lehmann-Str. 43 01139 Dresden
11854,Fahrschule Nord Gernot Krakowsky,Dresden,0351 8022221,01099,Bautzner Str. 44 01099 Dresden


### Output in Excel and the download Link


In [255]:
df.to_excel('Driving_Schools_Germany.xlsx', index=False)
FileLink(r'Driving_Schools_Germany.xlsx')

C:\Users\Besitzer\Car Industry Data Scraping\Driving_Schools_Germany.xlsx